# Prompting techniques from https://www.promptingguide.ai/techniques

In [7]:
from dotenv import load_dotenv
from google import genai
from google.genai import types
load_dotenv() 

# Creating the client
# The client gets the API key from the environment variable `GEMINI_API_KEY`.
client = genai.Client()

In [9]:
# Simple test
response = client.models.generate_content(
    model="gemini-2.5-flash", contents="Explain how AI works in a few words"
)
print(response.text)

AI learns patterns from data to make decisions or predictions.


In [14]:
# Zero-shot prompting: No example or demonstration to the LLM
response = client.models.generate_content(
    model="gemini-2.5-flash", 
    contents="Classify the text into neutral, negative or positive: Text: The time off was ok. Sentiment:"
)
print(response.text)

**Positive** (though very mildly so, leaning towards acceptable/satisfactory rather than enthusiastic).

"Ok" indicates that it was acceptable and not negative, implying a basic level of satisfaction.


In [5]:
# Few-shot Enable in-context learning providing demonstration in the
# prompt to steer the model to better performance
prompt = '''
    Classify the sentiment of the text below as **Positive**, **Negative**, or **Neutral**, based on the example format.
    Example:
    <query>
    The time off was ok
    </query>
    <response>
    Neutral
    </response>

    Classify:
    <query>
    I loved that food
    </query>
'''
response = client.models.generate_content(
    model="gemini-2.5-flash", 
    contents=prompt
)
print(response.text)


<response>
Positive
</response>


In [11]:
# Chain-of-Thought enable reasoning before answering. It can be combined with few shot.
# Used in ReAct
prompt = '''
The content below are examples to reason before answering.
Table: João - 2 sales | Maria - 5 sales | Pedro - 3 sales
Question: Who sold more and sales average?
Thinking step by step.
1. Comparing: Maria (5) > Pedro (3) > João (2)
2. Sales sum: 2 + 3 + 5 = 10
3. Average: 10 / 3 = 3,33 (prox)
Answer: Maria sold more; Average was 3,33.

Question: The laptop is not turning on and it's connected to the energy.
Let's think step by step.

Answer the next question below:
'''
question = '''
Question: Which Brazilian states don't have a letter "a" in the name?
'''
response = client.models.generate_content(
    model="gemini-2.5-flash", 
    contents=prompt + question
)
print(response.text)

Let's think step by step.

1.  **Recall/List all Brazilian states:**
    *   Acre
    *   Alagoas
    *   Amapá
    *   Amazonas
    *   Bahia
    *   Ceará
    *   Espírito Santo
    *   Goiás
    *   Maranhão
    *   Mato Grosso
    *   Mato Grosso do Sul
    *   Minas Gerais
    *   Pará
    *   Paraíba
    *   Paraná
    *   Pernambuco
    *   Piauí
    *   Rio de Janeiro
    *   Rio Grande do Norte
    *   Rio Grande do Sul
    *   Rondônia
    *   Roraima
    *   Santa Catarina
    *   São Paulo
    *   Sergipe
    *   Tocantins

2.  **Examine each state name for the presence of the letter "a" (case-insensitive):**
    *   Acre: Contains "A"
    *   Alagoas: Contains "A"
    *   Amapá: Contains "a"
    *   Amazonas: Contains "a"
    *   Bahia: Contains "a"
    *   Ceará: Contains "a"
    *   Espírito Santo: Contains "a"
    *   Goiás: Contains "a"
    *   Maranhão: Contains "a"
    *   Mato Grosso: Contains "a"
    *   Mato Grosso do Sul: Contains "a"
    *   Minas Gerais: Contai

In [10]:
# Without Chain-of-Thought
question = '''
Question: Which Brazilian states don't have a letter "a"?
'''
response = client.models.generate_content(
    model="gemini-2.5-flash", 
    contents=question
)
print(response.text)

There are two Brazilian states that don't have the letter "a" in their names:

1.  **Rondônia**
2.  **Sergipe**


# Meta prompting
Focuses on structure and syntactical aspects of tasks and 
problems rather than specific content details.
The goal is to construct a more abstract, structured way of interacting with LLM.

# Characteristics
1. Structure-oriented: Prioritizes the format and pattern of problems and solutions over specific content.
2. Syntax-focused: Uses syntax as a guiding template for the expected response or solution.
3. Abstract examples: Employs abstracted examples as frameworks, illustrating the structure of problems and solutions without focusing on specific details.
4. Versatile: Applicable across various domains, capable of providing structured responses to a wide range of problems.
5. Categorical approach: Draws from type theory to emphasize the categorization and logical arrangement of components in a prompt.





In [6]:
# Meta prompting
instruction = '''
You are a prompt engineer. Your task is to generate a clear and effective prompt for an AI language model that will help create a concise technical explanation of a given software architecture concept. The explanation should follow this structure:

1. A brief definition (2-3 sentences)
2. A real-world analogy to simplify the concept
3. A short example or use case (code optional)

The target audience is intermediate software developers. Use simple language, avoid unnecessary jargon, and make the result no longer than 300 words.
'''
query = "Generate a prompt for the concept: **Event-Driven Architecture**."
response = client.models.generate_content(
    model="gemini-2.5-flash", 
    contents=instruction + query
)
print(response.text)

Okay, here is the prompt designed for the AI language model:

---

**Prompt for AI Language Model:**

You are a technical writer specializing in software architecture. Your task is to generate a concise explanation of the software architecture concept: **Event-Driven Architecture**.

The explanation is for intermediate software developers. Use simple language and avoid unnecessary jargon.

Follow this exact structure:

1.  **Definition:** Provide a brief, 2-3 sentence definition of Event-Driven Architecture.
2.  **Analogy:** Include a clear, real-world analogy to simplify the concept.
3.  **Example/Use Case:** Offer a short example or common use case where Event-Driven Architecture would be beneficial. Code is optional.

Keep the entire explanation under 300 words. Aim for clarity and immediate understanding.


# Self-consistency
Aims to replace the naive greedy decoding used in chain-of-thought prompting. Sampel multiple, diverse reasoning paths throught few-shot CoT, and use the generations to select the most consistent answer.

Let's repeat the prompt with 1 temperature 5-10 times and get the best result

In [13]:
# Self-consistency
from collections import Counter

instruction = '''
You are a software architect. Think step by step and estimate how many microservices might be needed for an e-commerce platform that includes user management, product catalog, inventory, payment, and order tracking.

Explain your reasoning before stating the final number of microservices.
'''
results = []
for _ in range(5): 
    response = client.models.generate_content(
        model="gemini-2.5-flash",
        contents=instruction,
        config=types.GenerateContentConfig(
            temperature=1.0,
        ),
    )
    answer = response.text.strip()
    results.append(answer)

most_common = Counter(results).most_common(1)
print(f"Most common answer:\n{most_common[0][0]}")
print(results)

Most common answer:
As a software architect, estimating the number of microservices for a complex platform like e-commerce involves identifying distinct functional boundaries, considering scalability, independent deployment, and team autonomy. My approach is to break down the platform into its core business capabilities (bounded contexts), and then identify supporting cross-cutting concerns.

---

### Step-by-Step Reasoning for Microservices Estimation

**1. Identify Core Business Capabilities (Bounded Contexts):**

These are the primary domains that represent distinct areas of the business, each with its own data and logic.

*   **User Management:**
    *   **Scope:** User registration, login (authentication), profile management (address, preferences), roles, permissions (authorization).
    *   **Why a Microservice?** Handles sensitive user data, authentication workflows, and is foundational to most interactions. Needs to be highly available and secure.
    *   **Proposed Service:** 

# Prompt Chaining
Break tasks into smaller tasks, once subtasks have been identified, the LLM is prompted with a subtask and the response is used as input to another prompt.

(Prompt) -> [LLM] -> Response -> [LLM]


In [28]:
# Prompt chaining 1/2

query = '''
Please generate a list of 3 chapters for an eBook titled "Mastering Python". Each chapter should have a title and a brief description (1-2 sentences). Return the chapters in the following format, where each chapter is separated by `###`:

    Chapter 1: [Title]
    Description: [Brief description]

    Chapter 2: [Title]
    Description: [Brief description]
'''
response = client.models.generate_content(
    model="gemini-2.5-flash", 
    contents=query
)
print(response.text)

Chapter 1: Beyond the Basics: Pythonic Paradigms
Description: This chapter delves into Python's unique features like decorators, generators, and context managers, demonstrating how to write more efficient, elegant, and "Pythonic" code. It also covers advanced data structures and functional programming concepts.

###
Chapter 2: Object-Oriented Mastery & Design Patterns
Description: Explore advanced object-oriented programming (OOP) in Python, including inheritance, polymorphism, and abstract base classes. This chapter also introduces common design patterns and their Pythonic implementations for building robust and scalable applications.

###
Chapter 3: Performance, Concurrency, and Deployment
Description: Learn strategies for optimizing Python code performance, including profiling and C extensions. This chapter also covers concurrency with threads and asyncio, and essential considerations for deploying Python applications into production environments.


In [29]:
# Prompt chaining 2\2
chapters = response.text.split('###')

for chapter in chapters:
    # Extract title and description
    title, description = chapter.strip().split('\n')
    title = title.replace("Chapter", "").strip()
    description = description.replace("Description:", "").strip()
    query = f"""
    Write a detailed chapter about '{title}'. The chapter should cover the following:
    - Introduction to the topic.
    - Key concepts explained in simple terms.
    - Example use cases and applications for the concept.
    - Summary at the end of the chapter.

    Chapter description: {description}
    """
    content = client.models.generate_content(
        model="gemini-2.5-flash", 
        contents=query
    )
    
    print(f"Chapter Title: {title}\n")
    print(f"Chapter Description: {description}\n")
    print(f"Chapter Content: {content.text}\n")
    print('-==========-')


Chapter Title: 1: Beyond the Basics: Pythonic Paradigms

Chapter Description: This chapter delves into Python's unique features like decorators, generators, and context managers, demonstrating how to write more efficient, elegant, and "Pythonic" code. It also covers advanced data structures and functional programming concepts.

Chapter Content: # Chapter 1: Beyond the Basics: Pythonic Paradigms

## Introduction: The Spirit of Python

Welcome to "Beyond the Basics: Pythonic Paradigms"! You've likely mastered Python's fundamental syntax – variables, loops, functions, and basic data structures. But Python is more than just a programming language; it has a unique philosophy, a "way" of doing things that goes beyond mere correctness. This "way" is what we call **"Pythonic" code**.

Writing Pythonic code means embracing the language's idioms, best practices, and built-in features to produce code that is not just functional, but also:

*   **Readable:** Easy for humans to understand, often re

# Tree of Thoughts